# HuggingFace로 기계번역 구현하기

이번에는 HuggingFace로 기계번역 모델을 학습해 볼 것입니다.
먼저 필요한 library들을 설치하고 import합시다.

In [ ]:
!pip install transformers datasets evaluate accelerate scikit-learn sacrebleu sentencepiece

In [ ]:
import random
import evaluate
import numpy as np

from datasets import load_dataset
from transformers import AutoTokenizer

## Dataset 준비

기계번역을 위한 dataset을 준비하겠습니다.
Dataset은 `Helsinki-NLP/opus-100`을 활용합니다.

In [ ]:
books = load_dataset("Helsinki-NLP/opus-100", "en-ko")
books["train"][1]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/143k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/70.1M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/144k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'translation': {'en': "I ain't fishing' 'em out.", 'ko': '그거 꺼내려다가는'}}

보시다시피 각 data는 영어 문장을 `en`에, 한국어 문장을 `ko`에 저장하고 있습니다.

이번에는 tokenizer를 불러와 data를 미리 tokenize 하겠습니다.

In [ ]:
source_lang = "en"
target_lang = "ko"
prefix = "translate English to Korean: "
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-ko")

def preprocess_function(data):
    inputs = [prefix + text[source_lang] for text in data["translation"]]
    targets = [text[target_lang] for text in data["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

tokenized_books = books.map(preprocess_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/790k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/815k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/959k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

기계번역과 같이 입력과 출력을 모두 tokenize하는 경우에는 출력 text를 `tokenizer`의 `text_target`인자로 넘겨주면 됩니다. 이처럼 기계 번역 dataset을 불러오는 것은 imdb dataset과 별반 다르지 않습니다.

마지막으로 `data_collator`를 다음과 같이 구현합니다.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='google-t5/t5-small')

## Model 구현

이번에는 기계번역을 위한 sequence-to-sequence 모델을 구현하겠습니다.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained('google-t5/t5-small')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Sequence-to-sequence 모델은 위와 같이 `AutoModelForSeq2SeqLM`을 활용하면 됩니다.
Pre-trained 모델로 `google-t5/t5-small`이라는 것을 사용하고 있습니다.
이 코드 하나로 우리는 기계 번역과 같은 문제를 풀 수 있는 encoder-decoder 구조의 모델을 구현할 수 있습니다.

## 학습

마지막으로 학습 코드를 구현하겠습니다.
학습은 감정 분석과 똑같이 `training_args`를 정의하고 `Trainer`로 이전에 구현한 것들을 모두 수합하면 됩니다.

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="hf_mt",
    learning_rate=1e-3,
    per_device_train_batch_size=16,
    weight_decay=0.01,  # 첫 주차 때 배운 weight decay를 조절하는 hyper-parameter입니다.
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True  # 실제로 평가를 진행할 때는 내부적으로 정의된 beam search 등을 활용하여 text를 생성합니다.
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

마지막으로 학습하면 다음과 같습니다.

In [ ]:
trainer.train()
trainer.save_model()

Step,Training Loss
500,8.919200
1000,8.191500
1500,8.091500


KeyboardInterrupt: 

학습 결과를 보면 다음과 같습니다.

In [ ]:
from transformers import pipeline

text = "translate English to Korean: Hello! My name is Seunghyuk Cho."
translator = pipeline("translation", model='./hf_mt', max_new_tokens=128, device='cuda', tokenizer=tokenizer)
translator(text)

[{'translation_text': '-의 그 그 그 그 그 그 그 그 그 그 그 그...................................................'}]

보시다시피 성능이 그렇게 좋지 않습니다. 아마 더 많은 학습 시간을 요구하는 듯 합니다.

하지만 중요한 것은 data부터 모델, 학습 코드 구현까지 매우 쉽게 했다는 것입니다.

##과제 [4주차] HuggingFace로 두 문장의 논리적 모순 분류하기

In [ ]:
dataset = load_dataset("nyu-mll/glue", "mnli")

dataset


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [ ]:
# 학습 때는 train split만 활용하셔야 합니다. 나머지 split은 사용불가입니다.
train_dataset = dataset["train"]

# Validation data가 필요한 경우, train split에서 가져오셔야 합니다.
# train_test_split(test_size=0.2) 메서드 사용 테스트 20% 테스트용도로 사용
train_test_split = train_dataset.train_test_split(test_size=0.2)

# 학습데이타셋, 테스트 데이터 셋 분류
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

print(f"Train 데이터 개수: {len(train_dataset)}")
print(f"Test 데이터 개수: {len(test_dataset)}")

Train 데이터 개수: 314161
Test 데이터 개수: 78541


In [ ]:
# 데이터 전처fl
# Hugging Face에서 제공하는 사전 학습된 BERT 모델의 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 데이터셋을 토큰화하는 함수
# 입력: premise에 해당하는 문장과 hypothesis에 해당하는 문장 두 개가 입력으로 들어옵니다.
def preprocess_function(examples):
    return tokenizer(examples['premise'], examples['hypothesis'], truncation=True, padding='max_length', max_length=256)

# 토큰화된 데이터를 매핑
train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/314161 [00:00<?, ? examples/s]

Map:   0%|          | 0/78541 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# 사전 학습된 BERT 모델 로드 및 클래스 수 설정
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

# 훈련 인자 설정 (batch size, epoch 수 등)
training_args = TrainingArguments(
    output_dir="./results",          # 체크포인트가 저장될 디렉토리
    evaluation_strategy="epoch",     # 매 epoch마다 평가
    learning_rate=1e-3,              # 학습률 1e-3
    gradient_accumulation_steps=4,   # 배치 사이즈를 4개씩 모아 :속도 높이기 위해 설정
    per_device_train_batch_size=64,  # 각 GPU/CPU 장치당 배치 사이즈 : 현재 무료 코랩에서 32에서 정상 작동
    per_device_eval_batch_size=64,   # 평가 시 배치 사이즈
    num_train_epochs=1,              # 에포크 수 : 3 -> 1설정
    weight_decay=0.01,               # 가중치 감쇠
    logging_dir='./logs',            # 로그 저장 경로
    logging_steps=10,                # 로그 기록 주기
    fp16=True,  # Mixed Precision Training 활성화 : 메모리 사용량 줄이고 연산속도는 높여준다.
)




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
!pip install evaluate

In [ ]:
# 정확도 메트릭 로드 및 평가 함수 정의
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)


In [ ]:
# Trainer 설정
trainer = Trainer(
    model=model,                         # 학습할 모델
    args=training_args,                  # TrainingArguments 설정
    train_dataset=train_dataset,         # 학습 데이터셋
    eval_dataset=test_dataset,             # 평가 데이터셋
    compute_metrics=compute_metrics  # 정확도 메트릭 추가
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
# 모델 train
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
0,1.103400,1.098664


TrainOutput(global_step=1227, training_loss=1.1124683712785965, metrics={'train_runtime': 3538.0579, 'train_samples_per_second': 88.795, 'train_steps_per_second': 0.347, 'total_flos': 4.132354093468877e+16, 'train_loss': 1.1124683712785965, 'epoch': 0.9997962925239356})

In [ ]:
# 성능 평가
# 평가 결과 출력
eval_results = trainer.evaluate(eval_dataset=dataset['validation_matched'])

print(f"Validation Matched Accuracy: {eval_results['eval_accuracy'] * 100:.2f}%")



Validation Matched Accuracy: 35.45%


In [ ]:
# validation_matched 데이터셋에도 preprocess_function 적용
# dataset 객체에 'validation_matched' 키가 있다고 가정합니다.
# 만약 다른 키를 사용한다면, 해당 키로 변경해주세요.
if 'validation_matched' in dataset:
    dataset['validation_matched'] = dataset['validation_matched'].map(preprocess_function, batched=True)
else:
    print("Warning: 'validation_matched' key not found in dataset. Please check your dataset keys.")

# 성능 평가
# 평가 결과 출력
# dataset 객체에 'validation_matched' 키가 있다고 가정합니다.
# 만약 다른 키를 사용한다면, 해당 키로 변경해주세요.
if 'validation_matched' in dataset:
    eval_results = trainer.evaluate(eval_dataset=dataset['validation_matched'])
    print(f"Validation Matched Accuracy: {eval_results['eval_accuracy'] * 100:.2f}%")
else:
    print("Warning: 'validation_matched' key not found in dataset. Cannot perform evaluation.")

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Validation Matched Accuracy: 35.45%
